In [ ]:
import brightway2 as bw
from pathlib import Path
from gsa_framework import LCAModel
from gsa_framework.validation import Validation
from gsa_framework.utils_setac_lca import *
from gsa_framework.utils import read_hdf5_array
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
path_base = Path('/data/user/kim_a/setac_gsa/')
path_merlin = path_base / "merlin"
filepath_scores_lsa_dict = path_merlin / "scores_lsa_dict.pickle"
filepath_params_yes_0 = path_merlin / "params_yes_0.pickle"
filepath_params_yes_6 = path_merlin / "params_yes_6.pickle"

with open(filepath_scores_lsa_dict, 'rb') as f:
    scores_lsa_dict = pickle.load(f)
    
with open(filepath_params_yes_0, 'rb') as f:
    params_yes_0 = pickle.load(f)
    
with open(filepath_params_yes_6, 'rb') as f:
    params_yes_6 = pickle.load(f)

In [ ]:
seed = 3333
path_model_dir = path_base / "regression" / "{}_model".format(seed)
model, params_yes_xgboost, importance_dict = get_xgboost_params(path_model_dir, params_yes_0)

# Save rows cols, and tech_params

In [ ]:
bw.projects.set_current("GSA for setac")
co = bw.Database("CH consumption 1.0")
demand_act = [act for act in co if "Food" in act["name"]][0]
demand = {demand_act: 1}
method = ("IPCC 2013", "climate change", "GTP 100a")
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
print(lca.score)

tech_params_inf = lca.tech_params[params_yes_xgboost]
params_yes_xgboost_where = np.array([np.where(params_yes_0==p)[0][0] for p in params_yes_xgboost])

In [ ]:
filepath_row_acts = path_model_dir / 'row_acts.pickle'
filepath_col_acts = path_model_dir / 'col_acts.pickle'
filepath_tech_params = path_model_dir / 'tech_params.pickle'

filepath_row_acts_names = path_model_dir / 'row_acts_names.pickle'
filepath_col_acts_names = path_model_dir / 'col_acts_names.pickle'

In [ ]:
%%time
row_acts, col_acts = [],[]
for p in tech_params_inf:
    row = p['row']
    col = p['col']
    row_acts.append(bw.get_activity(lca.reverse_dict()[0][row]))
    col_acts.append(bw.get_activity(lca.reverse_dict()[0][col]))

In [ ]:
row_acts_names = []
col_acts_names = []
for i in range(len(row_acts)):
#     print('output: ' + col_acts[i]['name'] + ' (' + col_acts[i]['unit'] + ', ' + col_acts[i]['location'] + ')')
#     print('input:  ' + row_acts[i]['name'] + ' (' + row_acts[i]['unit'] + ', ' + row_acts[i]['location'] + ')\n')
    row_acts_names.append(
        [
            row_acts[i]['name'], row_acts[i]['unit'], row_acts[i]['location']
        ]
    )
    col_acts_names.append(
        [
            col_acts[i]['name'], col_acts[i]['unit'], col_acts[i]['location']
        ]
    )

In [ ]:
with open(filepath_row_acts, 'wb') as f:
     pickle.dump(row_acts, f)
with open(filepath_col_acts, 'wb') as f:
     pickle.dump(col_acts, f)       
with open(filepath_tech_params, 'wb') as f:
     pickle.dump(tech_params_inf, f)
        
with open(filepath_row_acts_names, 'wb') as f:
     pickle.dump(row_acts_names, f)
with open(filepath_col_acts_names, 'wb') as f:
     pickle.dump(col_acts_names, f)

In [ ]:
with open(filepath_row_acts, 'rb') as f:
     row_acts = pickle.load(f)
with open(filepath_col_acts, 'rb') as f:
     col_acts = pickle.load(f)       
with open(filepath_tech_params, 'rb') as f:
     tech_params_inf = pickle.load(f)
        
with open(filepath_row_acts_names, 'rb') as f:
     row_acts_names = pickle.load(f)
with open(filepath_col_acts_names, 'rb') as f:
     col_acts_names = pickle.load(f)

# Some analysis

In [ ]:
activity_dict = lca.activity_dict

In [ ]:
co.search("pork, fresh")

In [ ]:
input_acts = [
    co.search("hard cheese")[0],
    co.search("sausages")[0],
    co.search("beef")[0],
    co.search("chocolate")[0],
    co.search("pork, fresh")[0]
]

In [ ]:
plt.figure(figsize=(10,10))

G = nx.DiGraph()
pos = {
    activity_dict[demand_act]: (len(input_acts)//2+1,0),
    "bio": (len(input_acts)+1,0),# all biosphere
}

# First level
exchanges1 = []
exchanges1_acts = []
for exc in demand_act.exchanges():
    if exc['amount']!=0 and exc.get("type") != "production" and exc.input in input_acts:
        exchanges1.append(activity_dict[exc.input])
        exchanges1_acts.append(exc.input)
    
# Second level
exchanges2 = []
exchanges2_acts = []
for act in exchanges1_acts:
    exchanges2_temp = []
    exchanges2_acts_temp = []
    for exc in act.exchanges():
        if exc['amount']!=0 and exc.get("type") != "production":
            exchanges2_temp.append(activity_dict[exc.input])
            exchanges2_acts_temp.append(exc.input)
    exchanges2.append(exchanges2_temp)
    exchanges2_acts.append(exchanges2_acts_temp)
    
for i,exc in enumerate(exchanges):
    pos[exc] = (i+1,-1)

k = 1
for i,excs in enumerate(exchanges2):
    for j,exc in enumerate(excs):
        pos[exc] = (k,-2)
        k += 1
    k += 1

for key in pos.keys():
    G.add_node(key)

# Adjust label location
for key, val in pos.items():
    pos_labels[key] = (val[0],val[1]+0.1)
Gdraw = nx.draw_networkx_nodes(G, pos, node_size=100)
Gdraw = nx.draw_networkx_labels(G, pos_labels)

In [ ]:
exchanges2

In [ ]:
co.search("hard cheese")[0]

In [ ]:
n_inf = 60
tech_params_analysis = tech_params_inf[:n_inf]
row_acts_inds = tech_params_analysis["row"]
col_acts_inds = tech_params_analysis["col"]
importances = np.array(list(importance_dict.values()))
importances = importances // min(importances)
importances_graph = importances[:n_inf]

In [ ]:
G = nx.DiGraph()
for i in range(n_inf):
    G.add_edge(row_acts_inds[i], col_acts_inds[i], weight=importances_graph[i])
    
plt.figure(figsize=(10,10))
pos = nx.shell_layout(G)
nx.draw_networkx_nodes(G,pos,node_size=100)
nx.draw_networkx_edges(G,pos)
nx.draw_networkx_labels(G,pos)
edge_labels = { (u,v): "{:3.0f}".format(d['weight']) for u,v,d in G.edges(data=True) }
labels = nx.draw_networkx_edge_labels(G,pos,edge_labels=edge_labels)

In [ ]:
df = pd.DataFrame

In [ ]:
path_base

In [ ]:
def get_exc_inputs(acts, lca):
    lca.tech_params
    exc_inputs = {}
    for act in acts:
        for exc in act.exchanges():
            if exc["amount"]!=0 and exc["type"]!="production":
                exc_inputs[exc.input] = {
                    "amount": exc.amount,
                    "ind": activity_dict.get(exc.input, None)
                }
    return exc_inputs

In [ ]:
activity_name = "hard cheese"
for exc in demand_act.exchanges():
    if activity_name in exc.input["name"]:
        activity_amount = exc["amount"]
        break

In [ ]:
act1 = [
    (
        activity_dict.get(cheese_act), #in
        activity_dict.get(demand_act), #out
        activity_amount, #amount
    )
]
act2 = get_exc_inputs(act1)
act3 = get_exc_inputs(act2)
act4 = get_exc_inputs(act3)

In [ ]:
cheese3

In [ ]:
all_cheese = [cheese1, cheese2, cheese3, cheese4, ]
# for dict_act in all_cheese:
#     G.add_edge(acts_names_in[i], acts_names_out[i], weight=weights_rounded[i])

In [ ]:
all_cheese

In [ ]:
cheese_act

In [ ]:
G = nx.DiGraph()

n_display_characters = 20
weights  = uncertain_params['amount']
weights_rounded = abs(np.round(weights, decimals = 2))

acts_in, acts_out, acts_names_in, acts_names_out  = [0]*n_params, [0]*n_params, [0]*n_params, [0]*n_params

unique_nodes = {}

for i,p in enumerate(uncertain_params):
    acts_in[i]  = bw.get_activity(reverse_act_dict[p['row']])
    acts_out[i] = bw.get_activity(reverse_act_dict[p['col']])
    acts_names_in[i]  = str(p['row']) + '_' + acts_in[i]['name'][:n_display_characters]
    acts_names_out[i] = str(p['col']) + '_' + acts_out[i]['name'][:n_display_characters]
    if p['amount'] > 0:
        G.add_edge(acts_names_in[i], acts_names_out[i], weight=weights_rounded[i])
    elif p['amount'] < 0:
        G.add_edge(acts_names_out[i], acts_names_in[i], weight=weights_rounded[i])
    else:
        print('amount = 0')
        
#     node = unique_nodes.get(acts_names_in, None)
#     if node != None:
#         assert node == p['row']
#     unique_nodes[acts_names_in] = p['row']
    
edge_labels = { (u,v): d['weight'] for u,v,d in G.edges(data=True) }

In [ ]:
exc.as_dict()